In [1]:
import numpy as np
import itertools
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pymatgen.symmetry.bandstructure import HighSymmKpath

def latex_fix(label):
    replace = {
        "\Gamma": "Î“",
    }
    if label in replace:
        label = replace[label]
    return label

# Plotting of the Brillouin zone
def go_points(points, size=4, color="black", labels=None):
    mode = "markers" if labels is None else "markers+text"

    if labels is not None:
        for il in range(len(labels)):
            labels[il] = latex_fix(labels[il])

    import plotly.graph_objects as go
    return go.Scatter3d(
        x=[v[0] for v in points],
        y=[v[1] for v in points],
        z=[v[2] for v in points],
        marker=dict(size=size, color=color),
        mode=mode,
        text=labels,
        textfont_color=color,
        showlegend=False
    )


def go_line(v1, v2, color="black", width=2, mode="lines", text=""):
    import plotly.graph_objects as go
    return go.Scatter3d(
        mode=mode,
        x=[v1[0], v2[0]],
        y=[v1[1], v2[1]],
        z=[v1[2], v2[2]],
        line=dict(color=color),
        text=text,
        showlegend=False
    )

def plot_brillouin_zone(struc, fig=None):
    bz_lattice=struc.lattice.reciprocal_lattice
    if fig is None:
        fig = go.Figure()
#   Plot the three lattice vectors        
    vertex1 = bz_lattice.get_cartesian_coords([0.0, 0.0, 0.0])
    vertex2 = bz_lattice.get_cartesian_coords([1.0, 0.0, 0.0])
    fig.add_trace(go_line(vertex1, vertex2, color="green", mode="lines+text", text=["","a"]))
    vertex2 = bz_lattice.get_cartesian_coords([0.0, 1.0, 0.0])
    fig.add_trace(go_line(vertex1, vertex2, color="green", mode="lines+text", text=["","b"]))
    vertex2 = bz_lattice.get_cartesian_coords([0.0, 0.0, 1.0])
    fig.add_trace(go_line(vertex1, vertex2, color="green", mode="lines+text", text=["","c"]))

#   Plot the Wigner-Seitz cell
    bz = bz_lattice.get_wigner_seitz_cell()
    for iface in range(len(bz)):  # pylint: disable=C0200
        for line in itertools.combinations(bz[iface], 2):
            for jface in range(len(bz)):
                if (iface < jface
                    and any(np.all(line[0] == x) for x in bz[jface])
                    and any(np.all(line[1] == x) for x in bz[jface])):
                    fig.add_trace(go_line(line[0], line[1]))

#   Plot the path in the Brillouin zone
    kpath=HighSymmKpath(struc)
    
    for line in [[kpath.kpath["kpoints"][k] for k in p] for p in kpath.kpath["path"]]:
        for k in range(1, len(line)):
            vertex1 = line[k - 1]
            vertex2 = line[k]
            vertex1 = bz_lattice.get_cartesian_coords(vertex1)
            vertex2 = bz_lattice.get_cartesian_coords(vertex2)

            fig.add_trace(go_line(vertex1, vertex2, color="red"))

    labels=kpath.kpath["kpoints"]
    vecs = []
    for point in labels.values():
        vecs.append(bz_lattice.get_cartesian_coords(point))

    fig.add_trace(go_points(vecs, color="red", labels=list(labels.keys())))

    fig.update_layout(
        scene = dict(
            xaxis = dict(visible=False, range=[-1.15,1.15],),
            yaxis = dict(visible=False, range=[-1.15,1.15],),
            zaxis = dict(visible=False, range=[-1.15,1.15],),
        )
    )
    return fig

# Dealing with the bandstructures
def get_n_branch(bs):
    return len(bs.branches)

def get_n_band(bs):
    if not "phonon" in str(type(bs)):
        return bs.bands[list(bs.bands.keys())[0]].shape[0]
    else:
        return bs.bands.shape[0]
    
def get_branch_wavevectors(bs, i_branch):
    branch = bs.branches[i_branch]
    if not "phonon" in str(type(bs)):
        start_wavevector = bs.kpoints[branch['start_index']].frac_coords
        end_wavevector = bs.kpoints[branch['end_index']].frac_coords
    else:
        start_wavevector = bs.qpoints[branch['start_index']].frac_coords
        end_wavevector = bs.qpoints[branch['end_index']].frac_coords
    return start_wavevector, end_wavevector

def get_branch_labels(bs, i_branch):
    branch = bs.branches[i_branch]
    if not "phonon" in str(type(bs)):
        start_label = bs.kpoints[branch['start_index']].label
        end_label = bs.kpoints[branch['end_index']].label
    else:
        start_label = bs.qpoints[branch['start_index']].label
        end_label = bs.qpoints[branch['end_index']].label        
    start_label = latex_fix(start_label)
    end_label = latex_fix(end_label)
    return [start_label, end_label]

def get_branch_energies(bs, i_branch, i_band):
    branch = bs.branches[i_branch]
    i_start = branch['start_index']
    i_end = branch['end_index']
    if not "phonon" in str(type(bs)):
        energies = list(bs.bands.values())[0][i_band, i_start:i_end+1]
    else:
        energies = bs.bands[i_band, i_start:i_end+1]
    return energies

def get_branch_distances(bs, i_branch):
    branch = bs.branches[i_branch]
    i_start = branch['start_index']
    i_end = branch['end_index']
    distances= bs.distance[i_start:i_end+1]-bs.distance[i_start]*np.ones(i_end-i_start+1)    
    return distances
    
def get_plot_bs(bs, branch_list = "all", plot_range = [None,None]):
    if branch_list == "all":
        branch_list = range(get_n_branch(bs))

    if not "phonon" in str(type(bs)):
        yaxis_title = "E - E<sub>f</sub> (eV)"
        yshift = bs.get_vbm()['energy']
    else:
        yaxis_title = "Frequencies (THz)"
        yshift = 0.0

    if plot_range == [None,None]:
        band_list = range(get_n_band(bs))
    else:
        band_list = []
        for i_band in range(get_n_band(bs)):
            yvals = []
            for i_branch in branch_list:
                yvals.extend(get_branch_energies(bs, i_branch, i_band) - yshift)
            if plot_range[0] == None:
                if np.min(np.array(yvals) - plot_range[1])<=0:
                    band_list.append(i_band)
            elif plot_range[1] == None:
                if np.max(np.array(yvals) - plot_range[0])>=0:
                    band_list.append(i_band)
            else:
                if np.max(np.array(yvals) - plot_range[0])>=0 and np.min(np.array(yvals) - plot_range[1])<=0:
                    band_list.append(i_band)
        
    fig = go.Figure()

    labels = []
    for i_branch in branch_list:
        new_labels = get_branch_labels(bs, i_branch)
        new_xvals = get_branch_distances(bs, i_branch)
        if len(labels) == 0:
            labels.append(new_labels[0])
            xvals = new_xvals.tolist()
            tickvals = [new_xvals[0], new_xvals[-1]]
        elif labels[-1] != new_labels[0]:
            labels[-1] += "|" + new_labels[0]
            xvals.extend((new_xvals + tickvals[-1]).tolist())
            tickvals.append(new_xvals[-1] + tickvals[-1])
        else:
            xvals.extend((new_xvals + tickvals[-1]).tolist())
            tickvals.append(new_xvals[-1] + tickvals[-1])
        labels.append(new_labels[1])
    
    for tickval in tickvals[1:-1]:
        fig.add_vline(x=tickval, line_width=1, line_color="black")

    yvals_lowest = []
    for i_branch in branch_list:
        yvals_lowest.extend(get_branch_energies(bs, i_branch, band_list[0]) - yshift)
    yvals_highest = []
    for i_branch in branch_list:
        yvals_highest.extend(get_branch_energies(bs, i_branch, band_list[-1]) - yshift)

    if plot_range == [None,None]:
        yaxis_range = [
            np.min(yvals_lowest)-0.02*abs(np.min(yvals_lowest)),
            np.max(yvals_highest)+0.02*abs(np.max(yvals_highest))]
    elif plot_range[0] == None:
        yaxis_range = [np.min(yvals_lowest)-0.02*abs(np.min(yvals_lowest)), plot_range[1]]
    elif plot_range[1] == None:
        yaxis_range = [plot_range[0], np.max(yvals_highest)+0.02*abs(np.max(yvals_highest))]
    else:
        yaxis_range = [plot_range[0], plot_range[1]]
        
    for i_band in band_list:
        yvals = []
        for i_branch in branch_list:
            yvals.extend(get_branch_energies(bs, i_branch, i_band) - yshift)
            
        scatter = go.Scatter(x=xvals, y=yvals, mode="lines", name="band "+str(i_band+1))
        fig.add_trace(scatter)
    
    fig.update_layout(
        xaxis =  {'mirror': True, 'showgrid': False,
                  'ticks': 'inside',
                  'tickvals': tickvals,
                  'ticktext': labels,
                  'ticklen':0},
        yaxis =  {'mirror': True, 'showgrid': False, 'ticks': 'inside', 'ticklen':10},
        yaxis_range = yaxis_range,
        xaxis_title = "Wave Vector",
        yaxis_title = yaxis_title
    )
    
    return fig

def get_plot_dos(dos, plot_range = [None,None]):
    fig = go.Figure()
    if not "phonon" in str(type(dos)):
        xvals = dos.energies - dos.efermi
        xaxis_title = "E - E<sub>f</sub> (eV)"
        yvals = list(dos.densities.values())[0]
    else:
        xvals = dos.frequencies
        xaxis_title = "Frequencies (THz)"
        yvals = dos.densities

    scatter = go.Scatter(x=xvals, y=yvals, mode="lines")
    fig.add_trace(scatter)
    
    if plot_range == [None, None]:
        xaxis_range = [np.min(xvals)-0.02*abs(np.min(xvals)), np.max(xvals)+0.02*abs(np.max(xvals))]
        yaxis_range = [0, 1.02*np.max(yvals)]
    elif plot_range[0] == None:
        xaxis_range = [np.min(xvals)-0.02*abs(np.min(xvals)), plot_range[1]]
        i1 = np.argmin(abs(xvals-plot_range[1]))
        yaxis_range = [0, 1.02*np.max(yvals[:i1])]
    elif plot_range[1] == None:
        xaxis_range = [plot_range[0], np.max(xvals)+0.02*abs(np.max(xvals))]
        i0 = np.argmin(abs(xvals-plot_range[0]))
        yaxis_range = [0, 1.02*np.max(yvals[i0:])]
    else: 
        xaxis_range = [plot_range[0], plot_range[1]]
        i0 = np.argmin(abs(xvals-plot_range[0]))
        i1 = np.argmin(abs(xvals-plot_range[1]))
        yaxis_range = [0, 1.02*np.max(yvals[i0:i1])]
    
    fig.update_layout(
        xaxis =  {'mirror': True, 'showgrid': False, 'ticks': 'inside', 'ticklen':10},
        yaxis =  {'mirror': True, 'showgrid': False, 'ticks': 'inside', 'ticklen':10},
        xaxis_range = xaxis_range,
        yaxis_range = yaxis_range,
        xaxis_title = xaxis_title,
        yaxis_title = "DOS",
    )
    return fig


def get_plot_bs_and_dos(bs, dos, branch_list = "all", plot_range = [None,None]):
    # Bandstructure
    if branch_list == "all":
        branch_list = range(get_n_branch(bs))

    if not "phonon" in str(type(bs)):
        yaxis_title = "E - E<sub>f</sub> (eV)"
        yshift = bs.get_vbm()['energy']
    else:
        yaxis_title = "Frequencies (THz)"
        yshift = 0.0

    if plot_range == [None,None]:
        band_list = range(get_n_band(bs))
    else:
        band_list = []
        for i_band in range(get_n_band(bs)):
            yvals = []
            for i_branch in branch_list:
                yvals.extend(get_branch_energies(bs, i_branch, i_band) - yshift)
            if plot_range[0] == None:
                if np.min(np.array(yvals) - plot_range[1])<=0:
                    band_list.append(i_band)
            elif plot_range[1] == None:
                if np.max(np.array(yvals) - plot_range[0])>=0:
                    band_list.append(i_band)
            else:
                if np.max(np.array(yvals) - plot_range[0])>=0 and np.min(np.array(yvals) - plot_range[1])<=0:
                    band_list.append(i_band)
        
    fig = make_subplots(rows=1, cols=2, shared_yaxes=True, horizontal_spacing=0.01, column_widths=[4,1])

    labels = []
    for i_branch in branch_list:
        new_labels = get_branch_labels(bs, i_branch)
        new_xvals = get_branch_distances(bs, i_branch)
        if len(labels) == 0:
            labels.append(new_labels[0])
            xvals = new_xvals.tolist()
            tickvals = [new_xvals[0], new_xvals[-1]]
        elif labels[-1] != new_labels[0]:
            labels[-1] += "|" + new_labels[0]
            xvals.extend((new_xvals + tickvals[-1]).tolist())
            tickvals.append(new_xvals[-1] + tickvals[-1])
        else:
            xvals.extend((new_xvals + tickvals[-1]).tolist())
            tickvals.append(new_xvals[-1] + tickvals[-1])
        labels.append(new_labels[1])
    
    yvals_lowest = []
    for i_branch in branch_list:
        yvals_lowest.extend(get_branch_energies(bs, i_branch, band_list[0]) - yshift)
    yvals_highest = []
    for i_branch in branch_list:
        yvals_highest.extend(get_branch_energies(bs, i_branch, band_list[-1]) - yshift)

    if plot_range == [None,None]:
        yaxis_range = [
            np.min(yvals_lowest)-0.02*abs(np.min(yvals_lowest)),
            np.max(yvals_highest)+0.02*abs(np.max(yvals_highest))]
    elif plot_range[0] == None:
        yaxis_range = [np.min(yvals_lowest)-0.02*abs(np.min(yvals_lowest)), plot_range[1]]
    elif plot_range[1] == None:
        yaxis_range = [plot_range[0], np.max(yvals_highest)+0.02*abs(np.max(yvals_highest))]
    else:
        yaxis_range = [plot_range[0], plot_range[1]]
        
    for i_band in band_list:
        yvals = []
        for i_branch in branch_list:
            yvals.extend(get_branch_energies(bs, i_branch, i_band) - yshift)
        scatter = go.Scatter(x=xvals, y=yvals, mode="lines", name="band "+str(i_band+1))
        fig.add_trace(scatter, row=1, col=1)

    for tickval in tickvals[1:-1]:
        fig.add_vline(x=tickval, line_width=1, line_color="black", row=1, col=1)
    
    # DOS
    if not "phonon" in str(type(dos)):
        xvals2 = list(dos.densities.values())[0]
        yvals2 = dos.energies - dos.efermi
    else:
        xvals2 = dos.densities
        yvals2 = dos.frequencies

    scatter = go.Scatter(x=xvals2, y=yvals2, mode="lines", showlegend=False)
    fig.add_trace(scatter, row=1, col=2)

    i0 = np.argmin(abs(yvals2-yaxis_range[0]))
    i1 = np.argmin(abs(yvals2-yaxis_range[1]))
    xaxis2_range = [0, 1.02*np.max(xvals2[i0:i1])]

    fig.update_layout(
        xaxis =  {'mirror': True, 'showgrid': False,
                  'ticks': 'inside',
                  'tickvals': tickvals,
                  'ticktext': labels,
                  'ticklen':0},
        yaxis =  {'mirror': True, 'showgrid': False, 'ticks': 'inside', 'ticklen':10},
        yaxis_range = yaxis_range,
        xaxis_title = "Wave Vector",
        yaxis_title = yaxis_title,        
        xaxis2 =  {'mirror': True, 'showgrid': False, 'ticks': 'inside', 'ticklen':10},
        yaxis2 =  {'mirror': True, 'showgrid': False, 'ticks': 'inside', 'ticklen':10},
        xaxis2_range = xaxis2_range,
        xaxis2_title = "DOS",
    )
    
    return fig

<>:9: SyntaxWarning: invalid escape sequence '\G'
<>:9: SyntaxWarning: invalid escape sequence '\G'
/var/folders/m0/79fdh5rs75s90y90257qzxg40000gn/T/ipykernel_95958/265495156.py:9: SyntaxWarning: invalid escape sequence '\G'
  "\Gamma": "Î“",


In [2]:
import numpy as np
from mp_api.client import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.phonon.plotter import PhononBSPlotter
from jupyter_jsmol.pymatgen import quick_view
#from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [3]:
mp_key = "g2nCFD5rMkPRt9qdpOGhbfHJf2mgbv5x"
mp_id = "mp-14116"

In [4]:
with MPRester(mp_key) as m:
    prim_struc = m.get_structure_by_material_id(mp_id)
    el_bs = m.get_bandstructure_by_material_id(mp_id)
    el_dos = m.get_dos_by_material_id(mp_id)
    ph_bs = m.get_phonon_bandstructure_by_material_id(mp_id)
    ph_dos = m.get_phonon_dos_by_material_id(mp_id)
conv_struc = SpacegroupAnalyzer(prim_struc).get_conventional_standard_structure()
symmops = SpacegroupAnalyzer(conv_struc).get_space_group_operations()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

# Structure symmetry

In [5]:
i_atom = 7
i_symmop = 2

In [6]:
view = quick_view(prim_struc, "packed", conventional = True)
display(view)

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [7]:
view.script('select antimony; color lightsalmon')
view.script('select sodium; color gold')
view.script('select lithium; color palegreen')
view.script('draw SYMOP ' + str(i_symmop) + ' {atomno = ' + str(i_atom) + '}')

In [8]:
symmop = symmops[i_symmop - 1]
print(symmop)

Rot:
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]


In [9]:
pos_init = conv_struc.sites[i_atom -1].frac_coords
print(pos_init)

[0.66666667 0.33333333 0.22617433]


In [10]:
pos_final = symmop.operate(pos_init)
print(pos_final)

[-0.66666667 -0.33333333 -0.22617433]


In [11]:
for k_atom, site in enumerate(conv_struc.sites):
    if np.linalg.norm((site.frac_coords - pos_final)%1) < 1e-6:
        j_atom = k_atom + 1
print(j_atom, conv_struc.sites[j_atom -1].frac_coords)

NameError: name 'j_atom' is not defined

# Brillouin zone

In [ ]:
import itertools
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Plotting of the Brillouin zone
def go_points(points, size=4, color="black", labels=None):
    mode = "markers" if labels is None else "markers+text"

    if labels is not None:
        for il in range(len(labels)):
            labels[il] = latex_fix(labels[il])

    import plotly.graph_objects as go
    return go.Scatter3d(
        x=[v[0] for v in points],
        y=[v[1] for v in points],
        z=[v[2] for v in points],
        marker=dict(size=size, color=color),
        mode=mode,
        text=labels,
        textfont_color=color,
        showlegend=False
    )

def go_line(v1, v2, color="black", width=2, mode="lines", text=""):
    import plotly.graph_objects as go
    return go.Scatter3d(
        mode=mode,
        x=[v1[0], v2[0]],
        y=[v1[1], v2[1]],
        z=[v1[2], v2[2]],
        line=dict(color=color),
        text=text,
        showlegend=False
    )

In [ ]:
struc = el_bs.structure
bz_lattice=struc.lattice.reciprocal_lattice
bz = bz_lattice.get_wigner_seitz_cell()
fig = go.Figure()
for iface in range(len(bz)):  # pylint: disable=C0200
        for line in itertools.combinations(bz[iface], 2):
            for jface in range(len(bz)):
                if (iface < jface
                    and any(np.all(line[0] == x) for x in bz[jface])
                    and any(np.all(line[1] == x) for x in bz[jface])):
                    fig.add_trace(go_line(line[0], line[1]))
fig.update_layout(
    scene = dict(
        xaxis = dict(visible=False, range=[-1.15,1.15],),
        yaxis = dict(visible=False, range=[-1.15,1.15],),
        zaxis = dict(visible=False, range=[-1.15,1.15],),
    )
)
fig.show()

In [ ]:
plot_brillouin_zone(el_bs.structure)

/opt/anaconda3/lib/python3.12/site-packages/pymatgen/symmetry/bandstructure.py:181: UserWarning:

The input structure does not match the expected standard primitive! The path may be incorrect. Use at your own risk.



# Electronic bandstructure

In [ ]:
fig_el_bs = get_plot_bs(el_bs, plot_range=[-4,7])
fig_el_bs.show()

In [ ]:
fig_el_dos = get_plot_dos(el_dos, plot_range=[-4,7])
fig_el_dos.show()

In [ ]:
fig_el_bs_and_dos = get_plot_bs_and_dos(el_bs, el_dos, plot_range=[-4,7])
fig_el_bs_and_dos.show()

In [ ]:
xvals = fig_el_bs.to_dict()["data"][0]["x"]
yvals_vbm = fig_el_bs.to_dict()["data"][2]["y"]
yvals_cbm = fig_el_bs.to_dict()["data"][3]["y"]

In [ ]:
xvals_band_edges = []
yvals_band_edges = []
for i in el_bs.get_vbm()["kpoint_index"]:
    xvals_band_edges.append(xvals[i])
    yvals_band_edges.append(yvals_vbm[i])
for i in el_bs.get_cbm()["kpoint_index"]:
    xvals_band_edges.append(xvals[i])
    yvals_band_edges.append(yvals_cbm[i])

In [ ]:
for i in el_bs.get_vbm()["kpoint_index"]:
    scatter = go.Scatter(
        x = xvals_band_edges, y = yvals_band_edges,
        mode = "markers", marker = dict(color="black"),
        showlegend=False)
    fig_el_bs.add_trace(scatter)
fig_el_bs.update_layout(xaxis_range = [xvals[0], xvals[-1]])
fig_el_bs.show()

In [ ]:
el_bs.get_vbm()

{'band_index': defaultdict(list, {<Spin.up: 1>: [21]}),
 'kpoint_index': [111],
 'kpoint': <pymatgen.electronic_structure.bandstructure.Kpoint at 0x310627710>,
 'energy': 4.1447,
 'projections': {}}

In [ ]:
el_bs.get_cbm()

{'band_index': defaultdict(list, {<Spin.up: 1>: [22]}),
 'kpoint_index': [135],
 'kpoint': <pymatgen.electronic_structure.bandstructure.Kpoint at 0x3106ccc80>,
 'energy': 5.1028,
 'projections': {}}

In [ ]:
i_vbm = list(el_bs.get_vbm()['band_index'].values())[-1][-1]
i_cbm = list(el_bs.get_cbm()['band_index'].values())[-1][0]
print("VBM:", i_vbm, "CBM:", i_cbm)

VBM: 21 CBM: 22


# Phonon bandstructure

In [ ]:
fig_ph_bs = get_plot_bs(ph_bs)
fig_ph_bs.update_yaxes(rangemode="tozero")
fig_ph_bs.show()

In [ ]:
fig_ph_dos = get_plot_dos(ph_dos)
fig_ph_dos.show()

In [ ]:
fig_ph_bs_and_dos = get_plot_bs_and_dos(ph_bs, ph_dos)
fig_ph_bs_and_dos.update_yaxes(rangemode="tozero")
fig_ph_bs_and_dos.show()

# Distances in the Brillouin zone

In [ ]:
xvals = fig_el_bs.to_dict()["data"][0]["x"]
for i_branch, branch in enumerate(el_bs.branches):
    wavevectors = get_branch_wavevectors(el_bs, i_branch)
    v0 = np.dot(np.transpose(el_bs.structure.lattice.reciprocal_lattice.matrix), wavevectors[0])
    v1 = np.dot(np.transpose(el_bs.structure.lattice.reciprocal_lattice.matrix), wavevectors[1])
    print("{0:3d} {1:9.6f} {2:9.6f}".format(i_branch, np.linalg.norm(v1-v0), xvals[branch["end_index"]]-xvals[branch["start_index"]]))

  0  1.171316  1.171316
  1  0.734850  0.734850
  2  1.268996  1.268996
  3  0.549212  0.549212
  4  1.369347  1.369347
  5  0.634498  0.634498
  6  0.192020  0.192020
  7  1.098983  1.098983
  8  0.370697  0.370697


In [ ]:
xvals = fig_ph_bs.to_dict()["data"][0]["x"]
for i_branch, branch in enumerate(ph_bs.branches):
    wavevectors = get_branch_wavevectors(ph_bs, i_branch)
    v0 = np.dot(np.transpose(ph_bs.structure.lattice.reciprocal_lattice.matrix), wavevectors[0])
    v1 = np.dot(np.transpose(ph_bs.structure.lattice.reciprocal_lattice.matrix), wavevectors[1])
    print("{0:3d} {1:9.6f} {2:9.6f}".format(i_branch, np.linalg.norm(v1-v0), xvals[branch["end_index"]]-xvals[branch["start_index"]]))

  0  1.195675  1.195675
  1  0.749402  0.749402
  2  1.296426  1.296426
  3  0.557246  0.557246
  4  1.397615  1.397615
  5  0.648213  0.648213
  6  0.194719  0.194719
  7  1.122738  1.122738
  8  0.376063  0.376063


In [ ]:
el_bs.structure

Structure Summary
Lattice
    abc : 5.999814952680207 5.999814952680207 5.999815038346766
 angles : 30.29502661119012 30.29502661119012 30.295027638981125
 volume : 48.70483702747853
      A : 5.791359 -1.567782 0.0
      B : 5.791359 1.567782 0.0
      C : 5.366944 0.0 2.682106
    pbc : True True True
PeriodicSite: Cu (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Rh (8.475, 0.0, 1.341) [0.5, 0.5, 0.5]
PeriodicSite: O (1.822, 1.268e-17, 0.2883) [0.1075, 0.1075, 0.1075]
PeriodicSite: O (15.13, -8.917e-18, 2.394) [0.8925, 0.8925, 0.8925]

In [ ]:
ph_bs.structure

Structure Summary
Lattice
    abc : 5.909988782613398 5.909988782613398 5.909988782589603
 angles : 30.12029579372189 30.12029579372189 30.120295793439386
 volume : 46.052734728654684
      A : 5.707001892714572 -1.535609588133812 0.0
      B : 5.707001892714572 1.535609588133812 0.0
      C : 5.293808266410216 0.0 2.627462930056636
    pbc : True True True
PeriodicSite: Cu (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Rh (8.354, 0.0, 1.314) [0.5, 0.5, 0.5]
PeriodicSite: O (14.91, 4.122e-18, 2.345) [0.8924, 0.8924, 0.8924]
PeriodicSite: O (1.798, -4.122e-18, 0.2828) [0.1076, 0.1076, 0.1076]

In [ ]:
prim_struc

Structure Summary
Lattice
    abc : 5.960931135821216 5.960931482593984 5.960931377645338
 angles : 30.003024583773303 30.003013066932613 30.003023935543027
 volume : 46.91270234409874
      A : 2.96897574 -0.00820503 5.16892791
      B : 1.37255093 2.63267779 5.16892791
      C : -0.01357204 -0.00820503 5.96091028
    pbc : True True True
PeriodicSite: Cu (0.0, 0.0, 0.0) [-0.0, -0.0, 0.0]
PeriodicSite: Rh (2.164, 1.308, 8.149) [0.5, 0.5, 0.5]
PeriodicSite: O (3.864, 2.336, 14.55) [0.8928, 0.8928, 0.8928]
PeriodicSite: O (0.4638, 0.2804, 1.747) [0.1072, 0.1072, 0.1072]

# Specific heat

In [ ]:
temperatures = np.arange(0,1000,5)
R = 8.314
nat = len(prim_struc)
ph_cv = np.array([ph_dos.cv(temperatures[i]) for i in range(len(temperatures))])/(3*nat*R)

In [ ]:
fig = go.Figure()
scatter = go.Scatter(x=temperatures, y=ph_cv)
fig.add_trace(scatter)
fig.add_hline(y=1, line_width=2, line_color="red")
fig.update_layout(
    xaxis =  {'mirror': True, 'showgrid': False, 'ticks': 'inside', 'ticklen':10},
    yaxis =  {'mirror': True, 'showgrid': False, 'ticks': 'inside', 'ticklen':10},
    xaxis_title = "Temperature",
    yaxis_title = "C<sub>v</sub> /\,3N_{\!at}R$",        
)
fig.show()

<>:9: SyntaxWarning:

invalid escape sequence '\,'

<>:9: SyntaxWarning:

invalid escape sequence '\,'

/var/folders/m0/79fdh5rs75s90y90257qzxg40000gn/T/ipykernel_12473/702228885.py:9: SyntaxWarning:

invalid escape sequence '\,'

